In [ ]:
# This script is a general template for the biophysical simulations and estimations from:

# Platkiewicz, J, Saccomano, Z, McKenzie, S, English, D, & Amarasingham, A. Monosynaptic inference via finely-timed spikes. 
# J Comput Neurosci (2021). https://doi.org/10.1007/s10827-020-00770-5

# Adaptations for different figures are easily accomodated by parameters changes where specified.
# For general questions about this article, please contact: aamarasingham@ccny.cuny.edu
# For questions specifically about this code, please contact: zsaccomano@gradcenter.cuny.edu 


# Point estimation of theta_hat
def compthta(Rtemp,Ttemp,delta): # function for computing estimate  
    Sobs = len(np.intersect1d(Rtemp,Ttemp))
    if Sobs != 0:
        mxSpk = np.max(np.append(Rtemp,Ttemp))
        bEdges = np.arange(0,mxSpk,delta)
        bo = histc(Rtemp, bEdges) 
        refCounts = np.append(bo[0],0)
        w = np.floor(Ttemp/delta)
        Nr = refCounts[w.astype('int')]
        Nr = Nr[Nr!=0]/delta
        naive = Sobs - np.sum(Nr)
        thtahat = naive/(1-((1/len(Nr))*np.sum(Nr)))
    else:
        thtahat = 0
        naive = 0
    return thtahat, naive
def histc(X, bins):
    map_to_bins = np.digitize(X,bins)
    r = np.zeros(bins.shape)
    for i in map_to_bins:
        r[i-1] += 1
    return [r, map_to_bins]


from brian2 import *
start_scope()
# Simulation parameters
duration = 10000 # ms
Ntrial = 10000 # computes this many simulation in parallel to improve sample size
# Membrane parameters (These values (Lines 38-78) can be taken from Tables 1 or 2 in the journal article)
cm = 250*pF               # membrane capacitance
gm = 25*nS                # membrane conductance
tauPre = 20.06*ms         # membrane time constant
tauPost = 19.59*ms        # membrane time constant
El = -70*mV               # resting potential
Vr = El+10*mV             # reset value
refractory_period = 0*ms  # refractory period

# Adaptive threshold parameters
# -- Pyramidal cell
tauTh_pyr = 13.54*ms
Vi_pyr = -52.80*mV
ka_pyr = 5*mV
ki_pyr = ka_pyr/1.
alpha_pyr = 0.45 #ka_pyr/ki_pyr
Vt_pyr = Vi_pyr+7.84*mV
# -- Interneuron
tauTh_int = 0.22*ms
Vi_int = -59.27*mV
ka_int = 5*mV
ki_int = ka_int/.75
alpha_int = 0.77 # ka_int/ki_int
Vt_int = Vi_int+1.57*mV

# Biophysical background input parameters
tauI = 4.97*ms
tauIr = 8.365*ms
muI = -51.21*mV
sigmaI_pyr = 14.72*mvolt
sigmaI = 2.93*mvolt 
xmin = muI-10.7*mV
xmax = muI+10.7*mV
period = 11.95
Delta = period
# Monosynapse parameters
tauS = 3*ms # synaptic conductance time constant
Esyn = 0*mV # synapse reversal potential 
PSC = 60*pA*2 # post-synaptic current amplitude
g0 = 2.8*nS #PSC
latency = 1*ms # physiological value: 1.5 ms

#-Integrate-and-fire neuron models
eqs_ref = Equations('''
dV/dt = (-V+mu+sigmaI_pyr*I)/tauPre : volt
dtheta/dt = (-theta+Vt_pyr+alpha_pyr*(V-Vi_pyr)*int(V>=Vi_pyr))/tauTh_pyr : volt
dI/dt = -I/tauIr+(2/tauIr)**.5*xi : 1
mu : volt 
''')
eqs_targ = Equations('''
dV/dt = (-V+mu+sigmaI*I-g0/gm*gsyn*(V-Esyn))/tauPost : volt  
dtheta/dt = (-theta+Vt_int+alpha_int*(V-Vi_int)*int(V>=Vi_int))/tauTh_int : volt 
dgsyn/dt = -gsyn/tauS : 1
dI/dt = -I/tauI+(2/tauI)**.5*xi : 1
mu : volt (linked)
''')
#-----Specify the model
reference = NeuronGroup(Ntrial,model=eqs_ref,threshold='V>theta',reset='V=Vr',refractory=refractory_period,method='euler')
targetOff = NeuronGroup(Ntrial,model=eqs_targ,threshold='V>theta',reset='V=Vr',refractory=refractory_period,method='euler')
targetOn = NeuronGroup(Ntrial,model=eqs_targ,threshold='V>theta',reset='V=Vr',refractory=refractory_period,method='euler')

reference.run_regularly('''mu = xmin+(xmax-xmin)*rand()''',dt=period*ms)
targetOff.mu = linked_var(reference,'mu')
targetOn.mu = linked_var(reference,'mu')
#-----Parameter initialization
reference.V = (xmax-xmin)*rand()+xmin
reference.theta = Vt_pyr+alpha_pyr*(reference.V-Vi_pyr)*np.sign(reference.V-Vi_pyr)
reference.I = 2*rand()-1
targetOff.V = (xmax-xmin)*rand()+xmin
targetOff.theta = Vt_int+alpha_int*(targetOff.V-Vi_int)*np.sign(targetOff.V-Vi_int)
targetOff.gsyn = 0
targetOff.I = 2*rand()-1
targetOn.V = (xmax-xmin)*rand()+xmin
targetOn.theta = Vt_int+alpha_int*(targetOn.V-Vi_int)*np.sign(targetOn.V-Vi_int)
targetOn.gsyn = 0
targetOn.I = 2*rand()-1

# Define the synaptic for the condition target = on
synOn = Synapses(reference,targetOn,model='''w : 1''',
         on_pre='''
         gsyn += 1
         ''',method='Euler')    
synOn.connect(i=arange(Ntrial),j=arange(Ntrial))
synOn.delay = latency
synOn.w = 1 

# -----Specify which variables to record -----
offSpikes = SpikeMonitor(targetOff)
onSpikes= SpikeMonitor(targetOn)
refSpikes = SpikeMonitor(reference)

run(duration*ms) # run simulation for duration

# Collect spike times, concatenate spike times from "each" simulated neuron
# into one long trial.
R = np.sort(refSpikes.t/ms + duration*refSpikes.i)
Toff = np.sort(offSpikes.t/ms + duration*offSpikes.i)
Ton = np.sort(onSpikes.t/ms + duration*onSpikes.i)

# Compute CCGs
lgs = np.arange(-20,21,1)
C0 = np.zeros(len(lgs))
C1 = np.zeros(len(lgs))
for k in range(len(lgs)):
    C0[k] = len(np.intersect1d(np.round(Toff)-lgs[k],np.round(R)))
    C1[k] = len(np.intersect1d(np.round(Ton)-lgs[k],np.round(R)))

peak = lgs[C1==np.max(C1)]
# Round spike trains and shift R:
Toff = np.round(Toff)
Ton = np.round(Ton)
Rshift = np.round(R) + peak

# Compute Estimates
cardR = len(Rshift)
if cardR!=0:
    est,est2 = compthta(Rshift,np.round(Ton),Delta)
    thtahat = est
    sOn = len(np.intersect1d(np.round(Ton),Rshift))
    sOff = len(np.intersect1d(np.round(Toff),Rshift))
    thta = (sOn-sOff)
else:
    thtahat = np.nan
    thta = np.nan
    print('Invalid parameters were used.')

# Plot CCGs and Point Estimation
plt.axvline(x=peak, ymin=0, ymax=1,label='Peak Lag',c='k',linestyle=':')
plt.plot(lgs,C1,label='Monosynapse On',c='g')
plt.plot(lgs,C0,label='Monosynapse Off',c='r')

plt.scatter(peak,sOn-thtahat,label='Point Estimation',c='b')
plt.xlabel('Lag (ms)')
plt.ylabel('Synchrony')
plt.legend()
print('Observed CCG peak (Synapse=On) = ',sOn)
print('Observed CCG peak (Synapse=Off) = ',sOff)
print('Estimation of (Synapse=On)-(Synapse=Off) = ',np.round(sOn-thtahat))